In [2]:
import tensorflow as tf
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import numpy_indexed as npi
import pandas as pd
from true_state_viewer import TrueStateTreeGraphViz, display_tree_pairs

/usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


## True state data

Firstly, I plan to train a VAE on true states

I will create a new dataset of all of the unique true states observed and save it to a csv file

In [2]:
def convert_to_dataframe(np_true_state):
    column_dict = {}
    for node in range(13):
        column_dict[f"{node}_unknown"] = np_true_state[:,node*6+0]
        column_dict[f"{node}_known"] = np_true_state[:,node*6+1]
        column_dict[f"{node}_scanned"] = np_true_state[:,node*6+2]
        column_dict[f"{node}_none"] = np_true_state[:,node*6+3]
        column_dict[f"{node}_user"] = np_true_state[:,node*6+4]
        column_dict[f"{node}_privileged"] = np_true_state[:,node*6+5]
    
    dataset = pd.DataFrame(column_dict)
    return dataset

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
input_reader = JsonReader("logs/APPO/TrueStates_200_1000_Meander_small")


dfs = []
for e in range(1):
    print(f"loading {e}")
    data = input_reader.next()
    print(data)
#     print(data["pre_action_true_states"].shape)
#     print(data["obs"].shape)
#     print(data["pre_action_true_states"].shape)
    assert(data["obs"].shape[0] == data["pre_action_true_states"].shape[0])
    dfs.append(convert_to_dataframe(data["pre_action_true_states"]).drop_duplicates())
    dfs.append(convert_to_dataframe(data["blue_action_true_states"]).drop_duplicates())
    dfs.append(convert_to_dataframe(data["red_action_true_states"]).drop_duplicates())

print(f"concatenating")
dataset = pd.concat(dfs, ignore_index=True)

print(dataset.tail())

print(f"number of rows = {dataset.shape[0]}")
print("dropping duplicates...")

dataset = dataset.drop_duplicates()
print(f"number of rows = {dataset.shape[0]}")

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2023-02-03 11:05:11,708	WARNING json_reader.py:239 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs/APPO/TrueStates_200_1000_Meander_small/*.json', '/tf/Cage/Notebooks/logs/APPO/TrueStates_200_1000_Meander_small/*.zip']


loading 0
SampleBatch(100: ['obs', 'actions', 'prev_actions', 'rewards', 'prev_rewards', 'dones', 't', 'eps_id', 'unroll_id', 'agent_index', 'action_prob', 'action_logp', 'action_dist_inputs', 'obs_embeds', 'advantages', 'value_targets', 'pre_action_true_states', 'blue_action_true_states', 'red_action_true_states', 'afterstates'])


NameError: name 'convert_to_dataframe' is not defined

In [9]:
import os  
os.makedirs('csv_data', exist_ok=True)
dataset.to_csv('csv_data/all_true_states.csv')

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# Just to check it's worked
d2 = pd.read_csv('csv_data/all_true_states.csv')
# d2.compare(dataset)
d2.tail()


,Unnamed: 0.1,Unnamed: 0,0_unknown,0_known,0_scanned,0_none,0_user,0_privileged,1_unknown,1_known,...,11_scanned,11_none,11_user,11_privileged,12_unknown,12_known,12_scanned,12_none,12_user,12_privileged
4510,4510,707526,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
4511,4511,707634,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
4512,4512,707898,0,0,1,1,0,0,0,0,...,1,1,0,0,0,0,1,1,0,0
4513,4513,707901,0,0,1,1,0,0,0,0,...,1,0,1,0,0,0,1,1,0,0
4514,4514,710084,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1


In [7]:
# Run if loading
dataset = d2

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
dataset.tail()

,Unnamed: 0,0_unknown,0_known,0_scanned,0_none,0_user,0_privileged,1_unknown,1_known,1_scanned,...,11_scanned,11_none,11_user,11_privileged,12_unknown,12_known,12_scanned,12_none,12_user,12_privileged
4510,707526,0,0,1,1,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,1
4511,707634,0,0,1,1,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,1
4512,707898,0,0,1,1,0,0,0,0,1,...,1,1,0,0,0,0,1,1,0,0
4513,707901,0,0,1,1,0,0,0,0,1,...,1,0,1,0,0,0,1,1,0,0
4514,710084,0,0,1,1,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,1
